In [ ]:
import pandas as pd
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Input, Activation, concatenate, Bidirectional, GRU 
from keras import Model
from keras import optimizers
from tensorflow import keras

In [ ]:
data_full = pd.read_csv('Intrahour Volatility Dataset.csv')
shuffled_data_full = data_full.sample(frac=1, random_state=42)

In [ ]:
#We undergo the outlier removal and data reconstruction for both data sets

In [ ]:
#Train-test split
splitlimit = int(len(data_full)*0.8)
training_features_a, test_data_a = shuffled_data_full[:splitlimit], shuffled_data_full[splitlimit:]

training_features_b, test_data_b = data_full[:splitlimit], shuffled_data_full[splitlimit:]

In [ ]:
X_a = shuffled_data_full[["Return_Squared", "Hourly Volatility"]]
Y_a = shuffled_data_full["target"]
data_set_a = shuffled_data_full[["Return_Squared", "Hourly Volatility", "target"]]

X_b = data_full[["Return_Squared", "Hourly Volatility"]]
Y_b = data_full["target"]
data_set_b = data_full[["Return_Squared", "Hourly Volatility", "target"]]

In [ ]:
#Outlier Detection in training_data_features

training_features_a["hourly_volatility_rolling_median"] = training_features_a["Hourly Volatility"].rolling(window=41, center=True, min_periods=1).median()
training_features_a["return_squared_rolling_median"] = training_features_a["Return_Squared"].rolling(window=41, center=True, min_periods=1).median()
training_features_a["volatility minus median"] = (training_features_a["Hourly Volatility"] - training_features_a["hourly_volatility_rolling_median"]).abs()
training_features_a["return minus median"] = (training_features_a["Return_Squared"] - training_features_a["return_squared_rolling_median"]).abs()
volatility_outliers_removed_a = training_features_a[~(training_features_a['volatility minus median'] > 5 * training_features_a['volatility minus median'].median())]
both_outliers_removed_a = volatility_outliers_removed_a[~(volatility_outliers_removed_a['return minus median'] > 5 * volatility_outliers_removed_a['return minus median'].median())]


training_features_b["hourly_volatility_rolling_median"] = training_features_b["Hourly Volatility"].rolling(window=41, center=True, min_periods=1).median()
training_features_b["return_squared_rolling_median"] = training_features_b["Return_Squared"].rolling(window=41, center=True, min_periods=1).median()
training_features_b["volatility minus median"] = (training_features_b["Hourly Volatility"] - training_features_b["hourly_volatility_rolling_median"]).abs()
training_features_b["return minus median"] = (training_features_b["Return_Squared"] - training_features_b["return_squared_rolling_median"]).abs()
volatility_outliers_removed_b = training_features_b[~(training_features_b['volatility minus median'] > 5 * training_features_b['volatility minus median'].median())]
both_outliers_removed_b = volatility_outliers_removed_b[~(volatility_outliers_removed_b['return minus median'] > 5 * volatility_outliers_removed_b['return minus median'].median())]

In [ ]:
X_cleaned_a = both_outliers_removed_a[["Return_Squared", "Hourly Volatility"]]
Y_cleaned_a = both_outliers_removed_a["target"]
data_set_cleaned_a = both_outliers_removed_a[["Return_Squared", "Hourly Volatility", "target"]]

X_cleaned_b = both_outliers_removed_b[["Return_Squared", "Hourly Volatility"]]
Y_cleaned_b = both_outliers_removed_b["target"]
data_set_cleaned_b = both_outliers_removed_b[["Return_Squared", "Hourly Volatility", "target"]]

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler_a = MinMaxScaler()
training_data_features_scaled_a = scaler_a.fit_transform(X_cleaned_a)
data_set_scaled_a = scaler_a.fit_transform(data_set_cleaned_a)

scaler_b = MinMaxScaler()
training_data_features_scaled_b = scaler_b.fit_transform(X_cleaned_b)
data_set_scaled_b = scaler_b.fit_transform(data_set_cleaned_b)

In [ ]:
#Reconstructing 2 sets of training data 

backcandles = 15

Z_a = []

for j in range(2):
    Z_a.append([])
    for i in range(backcandles, training_data_features_scaled_a.shape[0]):
        Z_a[j].append(training_data_features_scaled_a[i-backcandles:i, j])
        
Z_a = np.moveaxis(Z_a, [0], [2])
Z_a, yi_a = np.array(Z_a), np.array(data_set_scaled_a[backcandles-1:, -1])
y_final_a = np.reshape(yi_a,(len(yi_a),1))
y_final_a = y_final_a[1:]


Z_b = []

for j in range(2):
    Z_b.append([])
    for i in range(backcandles, training_data_features_scaled_b.shape[0]):
        Z_b[j].append(training_data_features_scaled_b[i-backcandles:i, j])
        
Z_b = np.moveaxis(Z_b, [0], [2])
Z_b, yi_b = np.array(Z_b), np.array(data_set_scaled_b[backcandles-1:, -1])
y_final_b = np.reshape(yi_b,(len(yi_b),1))
y_final_b = y_final_b[1:]


In [ ]:
#LSTM on shuffled data model
lstm_input = Input(shape = (backcandles, 2), name = 'lstm_input')

inputs = LSTM(80, name='first_layer')(lstm_input)
inputs = Dense(1, name='dense_layer')(inputs)
output = Activation('sigmoid', name = 'output')(inputs)


model_a = Model(inputs = lstm_input, outputs = output)
model_a.compile(optimizer="adam", loss = "binary_crossentropy", metrics = ["accuracy"])
history_a = model_a.fit(x=Z_a, y=y_final_a, epochs = 30, validation_data = (Z_a, y_final_a))

In [ ]:
#LSTM on standard data model
lstm_input = Input(shape = (backcandles, 2), name = 'lstm_input')

inputs = LSTM(80, name='first_layer')(lstm_input)
inputs = Dense(1, name='dense_layer')(inputs)
output = Activation('sigmoid', name = 'output')(inputs)

model_b = Model(inputs = lstm_input, outputs = output)
model_b.compile(optimizer="adam", loss = "binary_crossentropy", metrics = ["accuracy"])

history_b = model_b.fit(x=Z_b, y=y_final_b, epochs = 30, validation_data = (Z_b, y_final_b))

In [ ]:
#FFNN on standard data model

input_shape = len(training_data_features_scaled_b[0, :]) 
ff_input = Input(shape=input_shape, name='ff_input')
inputs = Dense(80, activation='relu', name='first_layer')(ff_input)  
outputs = Dense(1, activation='sigmoid', name='output')(inputs)


model_c = Model(inputs=ff_input, outputs=outputs)
model_c.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

history_c = model_c.fit(x=training_data_features_scaled_b, y=data_set_scaled_b[:, -1], epochs=30, validation_data=(training_data_features_scaled_b, data_set_scaled_b[:, -1]))

In [ ]:
plt.figure(figsize=(9, 3))

plt.plot(history_b.history['accuracy'], color = 'red', label='LSTM on Ordered Data')

plt.plot(history_c.history['accuracy'], color = 'grey', label='FFNN on Ordered Data')

plt.plot(history_a.history['accuracy'], color = 'black', label='LSTM on Shuffled Data')
plt.gca().tick_params(axis='x', labelsize=15)
plt.gca().tick_params(axis='y', labelsize=15) 
plt.xlabel('Epoch', size = 17)
plt.ylabel('Training Accuracy', size = 17)
plt.legend(fontsize = 13)
plt.savefig('Shuffled Vs Ordered Models.jpg', format='jpg', dpi=300, bbox_inches='tight')
plt.show()